## 14.Transformer——模型训练与推理

**学习目标**

1. 能够编写数据加载与预处理代码

2. 熟悉Transformer模型的超参数设置

3. 能够用代码实现Transformer模型的训练

4. 能用代码实现Transformer模型的推理

****

Transformer 不仅在机器翻译上表现出色，还被广泛应用于其他 NLP 任务，如文本摘要、问答系统、文本分类等。自原始 Transformer 提出以来，出现了许多改进和变体，如 BERT（Bidirectional Encoder Representations from Transformers）、GPT（Generative Pre-trained Transformer）等，这些模型在不同的任务上都取得了显著的成果。

在这次课程中，我们将学习如何使用数据集来训练一个Transformer模型，并推理中英文翻译的任务。

****

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
import numpy as np
import math
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tools import transformer
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1. 超参数设置

In [25]:
d_model = 512          # 模型维度
num_heads = 8          # 多头注意力头数
num_layers = 6         # 编码器和解码器层数
d_ffn = 2048           # 前馈神经网络隐藏层维度
dropout = 0.1          # Dropout 概率
max_len = 50           # 最大序列长度
batch_size = 32        # 批大小
num_epochs = 50        # 训练轮数
lr = 1e-4               # 学习率

2. 自定义数据集类

In [26]:
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab, max_len):
        """
        初始化数据集类。

        参数:
        - src_sentences: 源语言句子列表（例如中文句子）。
        - tgt_sentences: 目标语言句子列表（例如英文句子）。
        - src_vocab: 源语言词汇表（字典，将单词映射到索引）。
        - tgt_vocab: 目标语言词汇表（字典，将单词映射到索引）。
        - max_len: 句子的最大长度（超过此长度的句子将被截断，不足的将被填充）。
        """
        self.src_sentences = src_sentences  # 源语言句子列表
        self.tgt_sentences = tgt_sentences  # 目标语言句子列表
        self.src_vocab = src_vocab  # 源语言词汇表
        self.tgt_vocab = tgt_vocab  # 目标语言词汇表
        self.max_len = max_len  # 句子的最大长度

    def __len__(self):
        """
        返回数据集中句子的总数。
        """
        return len(self.src_sentences)

    def __getitem__(self, idx):
        """
        根据索引返回一个数据样本。

        参数:
        - idx: 数据样本的索引。

        返回:
        - src_indices: 源语言句子的索引序列（转换为 PyTorch 张量）。
        - tgt_indices: 目标语言句子的索引序列（转换为 PyTorch 张量）。
        """
        src_sentence = self.src_sentences[idx]  # 获取源语言句子
        tgt_sentence = self.tgt_sentences[idx]  # 获取目标语言句子

        # 将句子转换为索引序列
        # 使用词汇表将单词映射为索引，如果单词不在词汇表中，则使用 '<unk>'（未知词）的索引
        src_indices = [self.src_vocab.get(word, self.src_vocab['<unk>']) for word in src_sentence.strip().split()]
        tgt_indices = [self.tgt_vocab.get(word, self.tgt_vocab['<unk>']) for word in tgt_sentence.strip().split()]

        # 填充或截断到固定长度
        # 使用 <pad>（填充词）将句子填充到 max_len，或截断超过 max_len 的部分
        src_indices = self.pad_or_truncate(src_indices, self.max_len, self.src_vocab['<pad>'])
        tgt_indices = self.pad_or_truncate(tgt_indices, self.max_len, self.tgt_vocab['<pad>'])

        # 将索引序列转换为 PyTorch 张量，并返回
        return torch.tensor(src_indices, dtype=torch.long), torch.tensor(tgt_indices, dtype=torch.long)

    def pad_or_truncate(self, sequence, max_len, pad_token):
        """
        将序列填充或截断到固定长度。

        参数:
        - sequence: 输入的索引序列。
        - max_len: 目标长度。
        - pad_token: 填充词的索引。

        返回:
        - 填充或截断后的序列。
        """
        if len(sequence) < max_len:
            # 如果序列长度小于 max_len，则在末尾填充 pad_token
            sequence = sequence + [pad_token] * (max_len - len(sequence))
        else:
            # 如果序列长度大于 max_len，则截断到 max_len
            sequence = sequence[:max_len]
        return sequence

3. 构建词汇表

In [27]:
def build_vocab(sentences, special_tokens=None):
    """
    构建词汇表。

    参数:
    - sentences: 句子列表，每个句子是一个字符串。
    - special_tokens: 特殊符号列表（如 '<pad>', '<unk>', '<sos>', '<eos>'），这些符号会被优先添加到词汇表中。

    返回:
    - vocab: 词汇表（字典，将单词映射到唯一的索引）。
    """
    # 使用 defaultdict 创建词汇表
    # lambda: len(vocab) 表示默认值为当前词汇表的长度（即新词的索引）
    vocab = defaultdict(lambda: len(vocab))

    # 如果有特殊符号，优先添加到词汇表中
    if special_tokens:
        for token in special_tokens:
            vocab[token]  # 将特殊符号添加到词汇表，索引为当前词汇表长度

    # 遍历所有句子，将句子中的单词添加到词汇表
    for sentence in sentences:
        for word in sentence.strip().split():  # 按空格分割句子为单词
            vocab[word]  # 将单词添加到词汇表，索引为当前词汇表长度

    # 返回构建好的词汇表
    return vocab

4. 定义加载数据集函数

In [28]:
def load_dataset(file_path):
    df = pd.read_csv(file_path, header=None)  # 没有列名，使用默认列名 0 和 1
    # 假设列 0 是中文句子，列 1 是英文句子
    src_sentences = df[0].tolist()  # 中文句子
    tgt_sentences = df[1].tolist()  # 英文句子
    return src_sentences, tgt_sentences

5. 数据加载与预处理

In [29]:
# 加载数据集
file_path = "./datasets/WMT-Chinese-to-English-Machine-Translation-newstest/damo_mt_testsets_zh2en_news_wmt18.csv"
src_sentences, tgt_sentences = load_dataset(file_path)  # 调用 load_dataset 函数加载源语言和目标语言句子

# 划分训练集和测试集（80% 训练集，20% 测试集）
# 使用 train_test_split 函数将数据集划分为训练集和测试集
# test_size=0.2 表示测试集占 20%，random_state=42 确保每次划分结果一致
src_train, src_test, tgt_train, tgt_test = train_test_split(
    src_sentences, tgt_sentences, test_size=0.2, random_state=42
)

# 构建词汇表
# 定义特殊符号列表，包括填充符、未知词、句子开始符和句子结束符
special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']
# 使用训练集的源语言句子构建源语言词汇表
src_vocab = build_vocab(src_train, special_tokens)  # 仅使用训练集构建词汇表
# 使用训练集的目标语言句子构建目标语言词汇表
tgt_vocab = build_vocab(tgt_train, special_tokens)  # 仅使用训练集构建词汇表

# 创建训练集和测试集
# 使用 TranslationDataset 类将训练集和测试集封装为 PyTorch 数据集
train_dataset = TranslationDataset(src_train, tgt_train, src_vocab, tgt_vocab, max_len)
test_dataset = TranslationDataset(src_test, tgt_test, src_vocab, tgt_vocab, max_len)

# 创建 DataLoader
# 使用 DataLoader 将数据集封装为可迭代的数据加载器
# batch_size 表示每个批次的样本数量，shuffle=True 表示打乱训练集数据
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# shuffle=False 表示不打乱测试集数据
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

6. 训练函数

In [30]:
def train(model, dataloader, optimizer, criterion, device):
    """
    训练模型的一个 epoch。

    参数:
    - model: 要训练的模型。
    - dataloader: 数据加载器，提供训练数据批次。
    - optimizer: 优化器，用于更新模型参数。
    - criterion: 损失函数，用于计算模型输出和目标之间的损失。
    - device: 设备（如 'cuda' 或 'cpu'），用于指定模型和数据的计算设备。

    返回:
    - 平均损失值（一个 epoch 的总损失除以批次数量）。
    """
    model.train()  # 将模型设置为训练模式
    total_loss = 0  # 初始化总损失

    # 遍历数据加载器中的每个批次
    for src, tgt in dataloader:
        # 将数据移动到指定设备（如 GPU）
        src, tgt = src.to(device), tgt.to(device)

        # 前向传播
        optimizer.zero_grad()  # 清空梯度缓存
        # 模型输入：源语言句子和目标语言句子（不包括最后一个词）
        output = model(src, tgt[:, :-1])  # 解码器输入不包括最后一个词
        # 计算损失：模型输出和目标语言句子（不包括第一个词）之间的交叉熵损失
        loss = criterion(output.reshape(-1, output.size(-1)), tgt[:, 1:].reshape(-1))
        
        # 反向传播和优化
        loss.backward()  # 计算梯度
        optimizer.step()  # 更新模型参数

        # 累加损失
        total_loss += loss.item()

    # 返回平均损失值
    return total_loss / len(dataloader)

7. 测试函数

In [31]:
def evaluate(model, dataloader, criterion, device):
    """
    评估模型性能。

    参数:
    - model: 要评估的模型。
    - dataloader: 数据加载器，提供评估数据批次。
    - criterion: 损失函数，用于计算模型输出和目标之间的损失。
    - device: 设备（如 'cuda' 或 'cpu'），用于指定模型和数据的计算设备。

    返回:
    - 平均损失值（总损失除以批次数量）。
    - BLEU 分数，用于评估翻译质量。
    """
    model.eval()  # 将模型设置为评估模式
    total_loss = 0  # 初始化总损失
    all_preds = []  # 保存所有预测的句子索引
    all_targets = []  # 保存所有目标句子索引

    # 禁用梯度计算，减少内存消耗并加速计算
    with torch.no_grad():
        # 遍历数据加载器中的每个批次
        for src, tgt in dataloader:
            # 将数据移动到指定设备（如 GPU）
            src, tgt = src.to(device), tgt.to(device)

            # 前向传播
            # 模型输入：源语言句子和目标语言句子（不包括最后一个词）
            output = model(src, tgt[:, :-1])
            # 计算损失：模型输出和目标语言句子（不包括第一个词）之间的交叉熵损失
            loss = criterion(output.reshape(-1, output.size(-1)), tgt[:, 1:].reshape(-1))
            total_loss += loss.item()  # 累加损失

            # 保存预测和目标句子
            # 使用 torch.argmax 获取模型预测的词索引
            preds = torch.argmax(output, dim=-1)
            # 将预测和目标句子的索引保存到列表中
            all_preds.extend(preds.cpu().numpy())  # 将张量转换为 NumPy 数组并保存
            all_targets.extend(tgt[:, 1:].cpu().numpy())  # 目标句子去掉第一个词

    # 计算 BLEU 分数
    # 使用 calculate_bleu 函数计算预测句子和目标句子之间的 BLEU 分数
    bleu_score = calculate_bleu(all_preds, all_targets, dataloader.dataset.tgt_vocab)

    # 返回平均损失值和 BLEU 分数
    return total_loss / len(dataloader), bleu_score

8. 计算 BLEU 分数

In [32]:
def calculate_bleu(preds, targets, tgt_vocab):
    """
    计算预测句子和目标句子之间的 BLEU 分数。

    参数:
    - preds: 预测句子的索引列表（每个句子是一个索引列表）。
    - targets: 目标句子的索引列表（每个句子是一个索引列表）。
    - tgt_vocab: 目标语言词汇表（字典，将单词映射到索引）。

    返回:
    - 平均 BLEU 分数。
    """
    # 将索引转换为单词
    # 创建索引到单词的映射字典
    idx_to_word = {idx: word for word, idx in tgt_vocab.items()}

    # 初始化 BLEU 分数列表
    bleu_scores = []

    # 使用平滑函数处理 BLEU 分数计算中的零值问题
    smoothing = SmoothingFunction().method1

    # 遍历预测和目标句子
    for pred, target in zip(preds, targets):
        # 将预测句子的索引转换为单词，并过滤掉填充符 '<pad>'
        pred_sentence = [idx_to_word[idx] for idx in pred if idx != tgt_vocab['<pad>']]
        # 将目标句子的索引转换为单词，并过滤掉填充符 '<pad>'
        target_sentence = [idx_to_word[idx] for idx in target if idx != tgt_vocab['<pad>']]

        # 计算当前句子的 BLEU 分数
        # sentence_bleu 计算单个句子的 BLEU 分数
        # [target_sentence] 表示将目标句子作为参考句子列表
        # smoothing_function 用于处理零值问题
        bleu_scores.append(sentence_bleu([target_sentence], pred_sentence, smoothing_function=smoothing))

    # 返回所有句子的平均 BLEU 分数
    return np.mean(bleu_scores)

9. 定义生成翻译函数

In [33]:
def generate_translation(model, src_sentence, src_vocab, tgt_vocab, max_len, device):
    """
    使用模型生成翻译结果。

    参数:
    - model: 训练好的翻译模型。
    - src_sentence: 源语言句子（字符串）。
    - src_vocab: 源语言词汇表（字典，将单词映射到索引）。
    - tgt_vocab: 目标语言词汇表（字典，将单词映射到索引）。
    - max_len: 生成句子的最大长度。
    - device: 设备（如 'cuda' 或 'cpu'），用于指定模型和数据的计算设备。

    返回:
    - 翻译后的目标语言句子（字符串）。
    """
    model.eval()  # 将模型设置为评估模式

    # 将源句子转换为索引序列
    # 使用源语言词汇表将单词映射为索引，未知词用 '<unk>' 的索引代替
    src_indices = [src_vocab.get(word, src_vocab['<unk>']) for word in src_sentence.strip().split()]
    # 将索引序列转换为 PyTorch 张量，并添加批次维度（unsqueeze(0)）
    src_indices = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)

    # 初始化目标句子（以 <sos> 开始）
    tgt_indices = [tgt_vocab['<sos>']]  # 开始符号

    # 逐词生成翻译结果
    for _ in range(max_len):
        # 将目标句子的索引序列转换为 PyTorch 张量，并添加批次维度
        tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long).unsqueeze(0).to(device)
        # 前向传播：将源句子和当前目标句子输入模型，得到输出
        output = model(src_indices, tgt_tensor)
        # 获取下一个词的索引（选择概率最大的词）
        next_word = torch.argmax(output[:, -1, :], dim=-1).item()

        # 检查索引是否有效
        if next_word >= len(tgt_vocab):
            next_word = tgt_vocab['<unk>']  # 使用未知词索引

        # 将下一个词添加到目标句子中
        tgt_indices.append(next_word)
        # 如果生成的是结束符号 <eos>，则停止生成
        if next_word == tgt_vocab['<eos>']:  # 结束符号
            break

    # 将索引转换为单词
    # 创建索引到单词的映射字典
    idx_to_word = {idx: word for word, idx in tgt_vocab.items()}
    # 将目标句子的索引序列转换为单词列表
    translation = [idx_to_word[idx] for idx in tgt_indices]
    # 去掉 <sos> 和 <eos>，并将单词列表拼接为字符串
    return " ".join(translation[1:-1])

10. 模型训练与推理

In [34]:
# 初始化模型、优化器和损失函数
# 创建 Transformer 模型实例
# 参数:
# - len(src_vocab): 源语言词汇表大小。
# - len(tgt_vocab): 目标语言词汇表大小。
# - d_model: 模型的维度（隐藏层大小）。
# - num_heads: 多头注意力机制的头数。
# - num_layers: 编码器和解码器的层数。
# - d_ffn: 前馈神经网络的隐藏层维度。
# - dropout: Dropout 概率。
# - max_len: 句子的最大长度。
model = transformer.Transformer(len(src_vocab), len(tgt_vocab), d_model, num_heads, num_layers, d_ffn, dropout, max_len).to(device)

# 创建 Adam 优化器
# 参数:
# - model.parameters(): 模型的可训练参数。
# - lr: 学习率。
optimizer = optim.Adam(model.parameters(), lr=lr)

# 创建损失函数（交叉熵损失）
# 参数:
# - ignore_index=src_vocab['<pad>']: 忽略填充符 '<pad>' 的损失计算。
criterion = nn.CrossEntropyLoss(ignore_index=src_vocab['<pad>'])

# 训练循环
for epoch in range(num_epochs):
    # 训练一个 epoch，并返回训练损失
    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    
    # 在测试集上评估模型，返回验证损失和 BLEU 分数
    val_loss, bleu_score = evaluate(model, test_dataloader, criterion, device)
    
    # 打印当前 epoch 的训练损失、验证损失和 BLEU 分数
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, BLEU: {bleu_score:.4f}")

# 保存模型
# 将模型的参数保存到文件 "transformer_model.pth" 中
torch.save(model.state_dict(), "transformer_model.pth")

Epoch [1/50], Train Loss: 7.9980, Val Loss: 7.6718, BLEU: 0.0052
Epoch [2/50], Train Loss: 7.3035, Val Loss: 7.3415, BLEU: 0.0098
Epoch [3/50], Train Loss: 6.8615, Val Loss: 7.0353, BLEU: 0.0117
Epoch [4/50], Train Loss: 6.3539, Val Loss: 6.5641, BLEU: 0.0151
Epoch [5/50], Train Loss: 5.8010, Val Loss: 6.1288, BLEU: 0.0204
Epoch [6/50], Train Loss: 5.2914, Val Loss: 5.7207, BLEU: 0.0279
Epoch [7/50], Train Loss: 4.8419, Val Loss: 5.3858, BLEU: 0.0384
Epoch [8/50], Train Loss: 4.4386, Val Loss: 5.0470, BLEU: 0.0505
Epoch [9/50], Train Loss: 4.0594, Val Loss: 4.7107, BLEU: 0.0630
Epoch [10/50], Train Loss: 3.6928, Val Loss: 4.4772, BLEU: 0.0796
Epoch [11/50], Train Loss: 3.3560, Val Loss: 4.2185, BLEU: 0.0971
Epoch [12/50], Train Loss: 3.0415, Val Loss: 3.9930, BLEU: 0.1130
Epoch [13/50], Train Loss: 2.7447, Val Loss: 3.8052, BLEU: 0.1276
Epoch [14/50], Train Loss: 2.4703, Val Loss: 3.5629, BLEU: 0.1437
Epoch [15/50], Train Loss: 2.2207, Val Loss: 3.4534, BLEU: 0.1571
Epoch [16/50], Trai

In [35]:
print("\n开始推理...")

# 输入中文句子
src_sentence = "你好，世界！"

# 调用 generate_translation 函数生成翻译结果
# 参数:
# - model: 训练好的翻译模型。
# - src_sentence: 输入的源语言句子（中文句子）。
# - src_vocab: 源语言词汇表（字典，将单词映射到索引）。
# - tgt_vocab: 目标语言词汇表（字典，将单词映射到索引）。
# - max_len: 生成句子的最大长度。
# - device: 设备（如 'cuda' 或 'cpu'），用于指定模型和数据的计算设备。
translation = generate_translation(model, src_sentence, src_vocab, tgt_vocab, max_len, device)

# 打印生成的翻译结果
print(f"生成翻译: {translation}")


开始推理...
生成翻译: heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat heat
